In [ ]:
import scipy.io
import numpy as np
import os
join = os.path.join

from PIL.Image import Image


import rectify
import cv06, cv07
import tools

In [ ]:
root_dir = '/home/hartvi/zs22/TDV/scene_1/'
img_dir = root_dir + 'images/'
Xs = np.load('allX.npy')
Ps = np.load("Ps.npy")
K = cv06.K
invK = cv06.invK


def squarifyP(P):
    return np.hstack([P, [0,0,0,1]])

def rel_P(P1, P2):
    return (squarifyP(Ps[k]) @ np.linalg.inv(squarifyP(Ps[i])))[:3, :]


task = [] # task variable for all data

for i in range(1, 13):
    for k in range(i, 13):
        imga = '{0:02}'.format(i)
        im_a = Image.load(join(img_dir, imga + '.jpg'))
        imgb = '{0:02}'.format(k)
        im_b = Image.load(join(img_dir, imgb + '.jpg'))
        Rt_ab = rel_P(Ps[i], Ps[k])
        F_ab = tools.calc_F(invK=invK, R=Rt_ab[:3, :3], t=Rt_ab[:, -1])

        u_a = tools.e2p(np.loadtxt(join(root_dir, "corresp", "u_{:02d}.txt".format(i))).T)
        u_b = tools.e2p(np.loadtxt(join(root_dir, "corresp", "u_{:02d}.txt".format(k))).T)
        corresp = np.loadtxt(
            join(root_dir, "corresp", "m_{:02d}_{:02d}.txt".format(i, k)), 
            dtype=int
        ).T
        u_a_p = u_a[:, corresp[0]]
        u_b_p = u_b[:, corresp[1]]
        inlier_indices = cv07.eval_inliers(u_a_p, u_b_p, F_ab, 1.1)
        u_a_i = u_a_p[:, inlier_indices]
        u_b_i = u_b_p[:, inlier_indices]
        
        [H_a, H_b, im_a_r, im_b_r] = rectify.rectify( F_ab, im_a, im_b )
        u_a_r = H_a @ u_a_i
        u_b_r = H_b @ u_b_i
        task_i = np.array( [ im_a_r, im_b_r, seeds ], dtype=object )
        task += [ task_i ]
        seeds = np.vstack( ( u_a_r[0,:], u_b_r[0], ( u_a_r[1] + u_b_r[1] ) / 2 ) ).T

IndexError: Replacement index 2 out of range for positional args tuple